In [1]:
from sympy import *
from sympy.physics.quantum import TensorProduct
from sympy import tensorcontraction
import numpy as np
init_printing()
import tensorflow as tf

In [2]:
def heaviside(v):
    if v > 0.0:
        return 1.0
    else:
        return 0.0

def signFunction(v):
    if v > 0.0:
        return 1.0
    else:
        return -1.0
    
def tensorToKelvinVector_2D(T):
    kv = np.zeros(4).reshape(4,1)
    kv[0] = T[0,0]
    kv[1] = T[1,1]
    kv[2] = T[2,2]
    kv[3] = 2**0.5*T[0,1]
    return kv

def tensorToKelvinMatrix_2D(T):
    km = np.zeros(16).reshape(4,4)
    km[0,0] = T[0,0,0,0]
    km[0,1] = T[0,0,1,1]
    km[0,2] = T[0,0,2,2]
    km[0,3] = 2**0.5*T[0,0,0,1]
    km[1,0] = T[1,1,0,0]
    km[1,1] = T[1,1,1,1]
    km[1,2] = T[1,1,2,2]
    km[1,3] = 2**0.5*T[1,1,0,1]
    km[2,0] = T[2,2,0,0]
    km[2,1] = T[2,2,1,1]
    km[2,2] = T[2,2,2,2]
    km[2,3] = 2**0.5*T[2,2,0,1]
    km[3,0] = 2**0.5*T[0,1,0,0]
    km[3,1] = 2**0.5*T[0,1,1,1]
    km[3,2] = 2**0.5*T[0,1,2,2]
    km[3,3] = 2*T[0,1,0,1]
    return km

In [3]:
# Symbols used
e_1, e_2, e_3 = symbols("epsilon_1,epsilon_2,epsilon_3", real=True)
# E, nu = symbols("E, nu", positive=True, real=True)
E = 1.e9
nu = 0.3
theta = symbols("theta", real=True)
v,g = symbols("v, g", real=True)
lmbda = E*nu/((1+nu)*(1-2*nu))
mu = E/(2*(1+nu))



#Coordinate
x_1 = [1,0,0]
x_2 = [0,1,0]
x_3 = [0,0,1]

x1_otimes_x1 = Matrix(tensorproduct(x_1,x_1)).reshape(3,3)
x1_otimes_x2 = Matrix(tensorproduct(x_1,x_2)).reshape(3,3)
x1_otimes_x3 = Matrix(tensorproduct(x_1,x_3)).reshape(3,3)
x2_otimes_x1 = Matrix(tensorproduct(x_2,x_1)).reshape(3,3)
x2_otimes_x2 = Matrix(tensorproduct(x_2,x_2)).reshape(3,3)
x2_otimes_x3 = Matrix(tensorproduct(x_2,x_3)).reshape(3,3)
x3_otimes_x1 = Matrix(tensorproduct(x_3,x_1)).reshape(3,3)
x3_otimes_x2 = Matrix(tensorproduct(x_3,x_2)).reshape(3,3)
x3_otimes_x3 = Matrix(tensorproduct(x_3,x_3)).reshape(3,3)

e = [e_1, e_2, e_3]

#Tangential stiffness tensor etc. in numpy array

C11 = lmbda*np.eye(3)+2*mu*x1_otimes_x1
C22 = lmbda*np.eye(3)+2*mu*x2_otimes_x2
C33 = lmbda*np.eye(3)+2*mu*x3_otimes_x3
C12 = mu*(x1_otimes_x2 + x2_otimes_x1)
C13 = mu*(x1_otimes_x3 + x3_otimes_x1)
C23 = mu*(x2_otimes_x3 + x3_otimes_x2)
C = np.array([[C11,C12,C13],[C12,C22,C23],[C13,C23,C33]])

eps =  np.array([[e_1,0,0],[0,e_2,0],[0,0,e_3]])

sig_iso_check = lmbda*(np.trace(eps))*np.eye(3) + 2*mu*eps

# transpose of fourth-order unit tensor, It:A=At
i5 = np.zeros(81).reshape(3,3,3,3)
i5[0,0,0,0]=1
i5[0,1,1,0]=1
i5[0,2,2,0]=1
i5[1,0,0,1]=1
i5[1,1,1,1]=1
i5[1,2,2,1]=1
i5[2,0,0,2]=1
i5[2,1,1,2]=1
i5[2,2,2,2]=1

# forth-order unit tensor, I:A=A, it has major symmetry but not supersymmetry
i6 = np.zeros(81).reshape(3,3,3,3)
i6[0,0,0,0]=1
i6[0,1,0,1]=1
i6[0,2,0,2]=1
i6[1,0,1,0]=1
i6[1,1,1,1]=1
i6[1,2,1,2]=1
i6[2,0,2,0]=1
i6[2,1,2,1]=1
i6[2,2,2,2]=1

# symmetric fourth-order unit tensor: I_sym : A = A_sym
I4 = np.zeros(81).reshape(3,3,3,3)
I4 = (i5 + i6)/2


# print(I4)
# sig_iso_check
# print(simplify(C))

sig_iso = np.tensordot(C,eps, axes=([2,3],[0,1]))
# print(sig_iso)

# C.reshape(3,3,3,3)
# print(C)
# print(I4)
# prod =np.einsum('ijmn,mnkl->ijkl', I4, C)

# I:C
C_I4 = np.tensordot(I4,C,2)
# print(C_I4 - C)

In [4]:
# Crack normal and tangent

n_1, n_2 = symbols("n_1,n_2", real=True)

ang = 0
pi = np.pi
rad = ang/180.0*pi
# n, m vectors
n = [np.sin(rad),-np.cos(rad),0]
m = [-n[1],n[0],0]

# n = [n_1,n_2,0]
# m = [-n_2,n_1,0]

n_otimes_n = np.array(tensorproduct(n,n))
n_otimes_m = np.array(tensorproduct(n,m))
m_otimes_n = np.array(tensorproduct(m,n))
m_otimes_m = np.array(tensorproduct(m,m))

NM = 0.5*(n_otimes_m + m_otimes_n)
NN = n_otimes_n
print(tensorToKelvinVector_2D(NM))
print(tensorToKelvinVector_2D(NN))

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.70710678]]
[[0.]
 [1.]
 [0.]
 [0.]]


In [5]:
# Forces resolved on the crack normal and tangent
# Check for the return mapping part line ca 155-174

friction_coef = 0.6
degradation = 0.622
H = degradation/(1-degradation)*C

eps_num =  np.array([[3.28039e-7,-1.621558e-8/2**0.5,0],[-1.621558e-8/2**0.5,-1.421485e-7,0],[0,0,0]])
eps_pl_num =  np.array([[0.0,0.0/2**0.5,0],[0.0/2**0.5,0.0,0],[0,0,0]])
eps_el_num = eps_num - eps_pl_num
sig = np.tensordot(C,eps_el_num, axes=([2,3],[0,1]))

Fp = sig - np.tensordot(H,eps_pl_num, axes=([2,3],[0,1]))

Fp_n_scalar = np.tensordot(Fp,NN)
Fp_nm_scalar = np.tensordot(Fp,NM)
yield_function = abs(Fp_nm_scalar)+friction_coef*heaviside(-Fp_n_scalar)*Fp_n_scalar

delta_gamma = (1 - degradation)/(2*mu)*yield_function

eps_pl_new = delta_gamma*signFunction(Fp_nm_scalar)*NM

print(tensorToKelvinVector_2D(Fp))
print(Fp_n_scalar, Fp_nm_scalar)
print(tensorToKelvinVector_2D(eps_pl_new))

[[359.58221154]
 [ -2.10048077]
 [107.24451923]
 [-12.47352308]]
-2.10048076923078 8.82011275297920
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-2.62682932e-09]]


In [6]:
# Tangential stiffness computation
# Line ca 176 - 215

delta_gamma = 6.919281e-18
eps_num =  np.array([[3.28039e-7,-1.621558e-8/2**0.5,0],[-1.621558e-8/2**0.5,-1.421485e-7,0],[0,0,0]])
eps_pl_num =  np.array([[0.0,-5.253464e-9/2**0.5,0],[-5.253464e-9/2**0.5,0.0,0],[0,0,0]])
eps_el_num = eps_num - eps_pl_num

sig = np.tensordot(C,eps_el_num, axes=([2,3],[0,1]))
Fp = sig - np.tensordot(H,eps_pl_num, axes=([2,3],[0,1]))
Fp_nm_scalar = np.tensordot(Fp,NM)
sign_Fp_nm = signFunction(Fp_nm_scalar)

eta = sign_Fp_nm*NM
C_nn = np.tensordot(C,NN, axes=([2,3],[0,1]))

dgamma_deps = (1-degradation)/(2*mu) * (sign_Fp_nm * 2 * mu * eta +  heaviside(-Fp_n_scalar)*friction_coef* C_nn)
deta_deps = 2*mu/abs(Fp_nm_scalar)*(I4 - np.tensordot(eta,eta,0))

second_term =  2*mu* np.tensordot(eta,dgamma_deps,0)
third_term = 2*mu*delta_gamma*deta_deps
D = C - second_term - third_term

print(eta)
# print('C_nn',C_nn)
# print(dgamma_deps)
# print(second_term)

# print(deta_deps)
# print(second_term)
# print(D)
# print('e',etaeta)

[[0 -0.500000000000000 0]
 [-0.500000000000000 0 0]
 [0 0 0]]


In [7]:
# print(tensorToKelvinVector_2D(eta))
# print('C_nn',tensorToKelvinVector_2D(C_nn))
# print(tensorToKelvinVector_2D(dgamma_deps))
# print(tensorToKelvinMatrix_2D(second_term))
# print(tensorToKelvinMatrix_2D(deta_deps))
# print(tensorToKelvinMatrix_2D(third_term))
print(tensorToKelvinMatrix_2D(D))
# print(tensorToKelvinMatrix_2D(C))

[[1.34615384e+09 5.76923077e+08 5.76923077e+08 0.00000000e+00]
 [5.76923077e+08 1.34615384e+09 5.76923077e+08 0.00000000e+00]
 [5.76923077e+08 5.76923077e+08 1.34615384e+09 0.00000000e+00]
 [9.25222027e+07 2.15885140e+08 9.25222027e+07 6.23846152e+08]]
